# IO

## 1. Paths

### 1.1. Normalize

In [ ]:
const path = require('path');

const dir = 'a/b/c/../d';

const normalizedDir = path.normalize(dir);
console.log(`* when path is "${dir}", then path.normalize(dir) is: "${normalizedDir}"`);

### 1.2. Join

In [ ]:
const path = require('path');

const dir1 = 'a/b/c';
const dir2 = '../x/y';
const fn = 'foo.txt';

const joinedPath = path.join(dir1, dir2, fn);
console.log(`* when "dir1=${dir1}", "dir2=${dir2}", "fn=${fn}" then "path.join(dir1, dir2, fn)" is: "${joinedPath}"`);

### 1.3. Absolute path

In [ ]:
const path = require('path');

const dir = '../a/b/c';

const absPath = path.resolve(dir);
console.log(`* when path is "${dir}", then path.resolve(dir) is: "${absPath}"`);

### 1.4. Relative path

In [ ]:
const path = require('path');

const dir1 = 'a/b/c/d/';
const dir2 = 'a/b/d/';

const re = path.relative(dir1, dir2);
console.log(`* relative path of "${dir1}" and "${dir2}" is: "${re}"`);

### 1.5. Name of directory

In [ ]:
const path = require('path');

const fullpath = 'a/b/c/d.txt';

const dirname = path.dirname(fullpath);
console.log(`* dirname of "${fullpath}" is: "${dirname}"`);

### 1.6. File name of path

In [ ]:
const path = require('path');

const fullpath = 'a/b/c/d.txt';

const basename = path.basename(fullpath);
console.log(`* basename of "${fullpath}" is: "${basename}"`);

### 1.7. File extenstion name

In [ ]:
const path = require('path');

const fullpath = 'a/b/c/d.txt';

const extname = path.extname(fullpath);
console.log(`* extname of "${fullpath}" is: "${extname}"`);

## 2. File operator

### 2.1. Remove file

In [ ]:
const fs = require('fs');

async function rm(filename) {
    return await fs.promises.unlink(filename);
}

### 2.2. Is file exist

In [ ]:
const fs = require('fs');
const path = require('path');

async function exist(filename) {
    try {
        await fs.promises.access(filename);
        return true;
    } catch (err) {
        return false;
    }
}

let dir = 'io.ipynb';
let ok = await exist('io.ipynb');
console.log(`* "${dir}" file exists? ${ok}`);

dir = path.resolve('../src');
ok = await exist(dir);
console.log(`* "${dir}" directory exists? ${ok}`);

### 2.3. Create empty file

In [ ]:
const fs = require('fs');

function touch(filename) {
    return new Promise((resolve, reject) => {
        fs.promises.open(filename, 'w')
            .then(h => fs.close(h.fd, () => resolve()))
            .catch(err => reject(err));
    });
}

try {
    await touch('test.txt');
    const ok = await exist('test.txt');
    console.log(`* after file created, "test.txt" exist? ${ok}`);
} finally {
    await rm('test.txt');
}

### 2.4. Create or remove empty folder

In [ ]:
const fs = require('fs');

async function mkdir(dirname, options = {}) {
    try {
        return await fs.promises.mkdir(dirname, options);
    } catch (err) {
        console.log(err.message);
    }
}

async function rmdir(dirname, options = {}) {
    try {
        return await fs.promises.rmdir(dirname, options);
    } catch (err) {
        console.log(err.message);
    }
}

await mkdir('test');

const ok = await exist('test');
console.log(`* after directory created, "test" exist? ${ok}`);

await rmdir('test');

### 3.5. Remove dirs

- Functions

In [ ]:
const fse = require('fs-extra');
const path = require('path');

function mkdirs(pathname) {
    return new Promise((resolve, reject) => {
        fse.mkdirs(pathname, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

function rmdirs(pathname) {
    return new Promise((resolve, reject) => {
        fse.remove(pathname, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

const dir = 'test/a/b';
const file = path.join(dir, 'c.txt');

await mkdirs(dir);
await touch(file);

const ok = await exist(file);
console.log(`* file "${file}" exist? ${ok}`);

await rmdirs('test');

## 3. Buffer

### 3.1. Convert with string

In [ ]:
const strSrc = 'Hello 大家好';

const bufSrc = Buffer.from(strSrc, 'utf-8');
const data = [...bufSrc];
console.log(`* convert string "${strSrc}" to bytes are: ${JSON.stringify(data)}`);

const bufDst = Buffer.from(data);
const strDst = bufDst.toString('utf-8');
console.log(`* convert data "${JSON.stringify(data)}" to string is: "${strDst}"`);

### 3.2. Concat

In [ ]:
const str1 = 'Hello ';
const str2 = '大家好';

const buf1 = Buffer.from(str1, 'utf-8');
const buf2 = Buffer.from(str2, 'utf-8');

const bufAll = Buffer.concat([buf1, buf2]);
const data = [...bufAll];

console.log(`* concat "${JSON.stringify([...buf1])}" and "${JSON.stringify([...buf2])}" is: ${JSON.stringify(data)}`);

const strAll = Buffer.from(data).toString('utf-8');
console.log(`* convert data "${JSON.stringify(data)}" to string is: "${strAll}"`);

### 3.3. Read write data

In [ ]:
const crypto = require('crypto');

let data = null;
let str = 'hello 大家好';
let sbuf = Buffer.from(str, 'utf-8');
let lbuf = Buffer.alloc(4);
lbuf.writeInt32BE(sbuf.byteLength, 0);

let cbuf = crypto.createHash('md5').update(sbuf).digest();

data = Buffer.concat([lbuf, sbuf, cbuf]);
console.log(`* the encoding data is: "${data.toString('hex')}"`);

const len = data.readInt32BE(0);
console.log(`* the data length is: ${len}`);

sbuf = data.slice(4, 4 + len);
cbuf = data.slice(4 + len);

str = sbuf.toString('utf-8');
console.log(`* the message string in buffer is: "${str}"`);

const checksum = crypto.createHash('md5').update(sbuf).digest();
console.log(`* calculate checksum is: ${checksum.toString('hex')} and validate checksum is: ${cbuf.toString('hex')}`);

### 3.4. Base64

In [ ]:
let base64 = '';

let str = 'hello 大家好';
let buf = Buffer.from(str, 'utf-8');

base64 = buf.toString('base64');
console.log(`* string ${str} to base64 is: "${base64}"`);

buf = Buffer.from(base64, 'base64');
str = buf.toString('utf-8');
console.log(`* base64 "${base64}" to string is: "${str}"`);

## 4. File read and write

### 4.1. Read and write file

In [ ]:
const fs = require('fs');

async function fwrite(filename, data) {
    return await fs.promises.writeFile(filename, data);
}

async function fappend(filename, data) {
    return await fs.promises.appendFile(filename, data);
}

async function fread(filename) {
    return await fs.promises.readFile(filename);
}

async function freadString(filename, encoding='utf8') {
    // @ts-ignore
    return await fs.promises.readFile(filename, encoding);
}

const filename = 'test.txt';

try {
    await fwrite(filename, Buffer.from('Hello', 'utf-8'));
    await fappend(filename, Buffer.from(' 大家好', 'utf-8'));

    const data = await fread(filename);
    console.log(`* read from file: "${data.toString('hex')}" and content is: "${data.toString('utf-8')}"`);

    const str = await freadString(filename);
    console.log(`* read from file: "${str}"`);
} finally {
    await rm(filename);
}

### 4.2. Read and write file from fd

In [ ]:
const fs = require('fs');

async function lfopen(filename, mode='w+') {
    return await fs.promises.open(filename, mode);
}

function lfclose(fd) {
    return new Promise((resolve, reject) => {
        fs.close(fd, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

function lfwrite(fd, buf) {
    return new Promise((resolve, reject) => {
         fs.write(fd, buf, (err, written) => {
             if (err) {
                 reject(err);
             } else {
                 resolve(written);
             }
         });
    });
}

function lfread(fd, buf, offset=0, length=-1, position=0) {
    if (length < 0) {
        length = buf.byteLength;
    }
    return new Promise((resolve, reject) => {
        fs.read(fd, buf, offset, length, position, (err, bytesRead) => {
            if (err) {
                reject(err);
            } else {
                resolve(bytesRead);
            }
        });
    });
}

const filename = 'test.txt';

try {
    const wbuf = Buffer.from('Hello 大家好', 'utf-8');

    const wfh = await lfopen(filename);
    const wlen = await lfwrite(wfh.fd, wbuf);
    console.log(`* file written, ${wlen} bytes was written`);
    lfclose(wfh.fd);

    const rfh = await lfopen(filename, 'r');
    const rbuf = Buffer.alloc(256);
    const rlen = await lfread(rfh.fd, rbuf);
    console.log(`* file read, ${rlen} bytes was read`);
    lfclose(rfh.fd);

    console.log(`* read content is: ${rbuf.toString('utf-8')}`);
} finally {
    await rm(filename);
}

### 4.3. Read and write from stream

In [ ]:
const fs = require('fs');

function swrite(filename, contents, encoding='utf-8') {
    return new Promise((resolve, reject) => {
        const ws = fs.createWriteStream(filename);
        console.log(`* write stream open? ${ws.writable}`);

        // @ts-ignore
        ws.setDefaultEncoding(encoding);
        
        ws.on('error', err => reject(err));
        ws.on('finish', () => {
            resolve(ws.bytesWritten);
            ws.close();
        });

        for (let content of contents) {
            ws.write(content);
        }
        ws.end();
    });
}

function sread(filename, encoding='utf-8') {
    return new Promise((resolve, reject) => {
        const ws = fs.createReadStream(filename);
        console.log(`* read stream open? ${ws.readable}`);
        
        const chunks = [];
        
        ws.on('data', chunk => chunks.push(chunk));
        ws.on('error', err => reject(err));
        ws.on('end', () => {
            const data = Buffer.concat(chunks);
            resolve(data.toString('utf-8'));
        });
    });
}

const filename = 'test.txt';

try {
    const len = await swrite(filename, ['Hello ', '大家好']);
    console.log(`* write finish, bytes to written: ${len}`)

    const str = await sread(filename);
    console.log(`* read finish, content is: "${str}"`);
} finally {
    await rm(filename);
}

### 4.4. Watch file

In [ ]:
const fs = require('fs');

function sleep(ms) {
    return new Promise(resolve => {
        setTimeout(() => resolve(), ms);
    });
}

const options = {
    recursive: true,
    encoding: 'utf-8'
};

// @ts-ignore
const watcher = fs.watch('.', options, (event, filename) => {
    console.log(`* ${filename} as been ${event}`);
});

watcher.on('change', (evnet, filename) => console.log(`* ${filename} trigger ${evnet} event`));

const filename = 'test.txt';
await touch(filename);
await fwrite(filename, Buffer.from('hello', 'utf-8'));
await fappend(filename, Buffer.from(' world', 'utf-8'));

await rm(filename);

await sleep(2000);
watcher.close();

## 5. Glob 

In [ ]:
const glob = require('glob');

function findFiles(pattern) {
    return new Promise((resolve, reject) => {
        glob(pattern, (err, files) => {
            if (err) {
                reject(err);
            } else {
                resolve(files);
            }
        });
    });
}

const files = await findFiles('./**/*.ipynb');
console.log(`* found files include: ${JSON.stringify([...files])}`);